# 知乎爬虫

In [1]:
import csv
import os
import pickle
import time

from scrapy import Selector
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from concurrent.futures import ThreadPoolExecutor
import threading

In [7]:
# 采用非常没有水平的手动扫码登入
# 不加载图片
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://www.zhihu.com/signin')

time.sleep(10)

In [3]:
url = 'https://www.zhihu.com/topics'

driver.get(url)

In [4]:
while True:
    try:
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div/div/div[2]/a[1]'))
        )
        more_button.click()
    except:
        break

selector = Selector(text=driver.page_source)

detail_url_list = selector.xpath('//div/div/div[2]/div/div/div/a[1]/@href').extract()

detail_url_list = ['https://www.zhihu.com' + url for url in detail_url_list]

with open('detail_url_list.pkl', 'wb') as f:
    pickle.dump(detail_url_list, f)

detail_url_list, len(detail_url_list)

(['https://www.zhihu.com/topic/19555513',
  'https://www.zhihu.com/topic/19551556',
  'https://www.zhihu.com/topic/19551137',
  'https://www.zhihu.com/topic/19551469',
  'https://www.zhihu.com/topic/19641262',
  'https://www.zhihu.com/topic/19551052',
  'https://www.zhihu.com/topic/19560585',
  'https://www.zhihu.com/topic/19554051',
  'https://www.zhihu.com/topic/19551805',
  'https://www.zhihu.com/topic/19552249',
  'https://www.zhihu.com/topic/19562435',
  'https://www.zhihu.com/topic/19628469',
  'https://www.zhihu.com/topic/19551167',
  'https://www.zhihu.com/topic/19555754',
  'https://www.zhihu.com/topic/19571906',
  'https://www.zhihu.com/topic/19557244',
  'https://www.zhihu.com/topic/19551151',
  'https://www.zhihu.com/topic/19559947',
  'https://www.zhihu.com/topic/19551387',
  'https://www.zhihu.com/topic/19570261',
  'https://www.zhihu.com/topic/19556311',
  'https://www.zhihu.com/topic/19551841',
  'https://www.zhihu.com/topic/19624425',
  'https://www.zhihu.com/topic/196

In [5]:
content_url = []

with open('./progress/detail_url_list.pkl', 'rb') as f:
    detail_url_list = pickle.load(f)
    
if os.path.exists('./progress/content_url.pkl'):
    with open('./progress/content_url.pkl', 'rb') as f:
        content_url = pickle.load(f)

# 保存进度的文件名
progress_file = './progress/progress_get_content.txt'

# 读取进度
if os.path.exists(progress_file):
    with open(progress_file, 'r') as f:
        start_url = f.read().strip()
else:
    start_url = None

# 在爬取每个URL之前，保存进度
for url in detail_url_list:
    if start_url is not None:
        if url == start_url:
            start_url = None
        continue

    # 爬取url的代码...
    driver.get(url)

    for _ in range(2):
        actions = ActionChains(driver)
        actions.send_keys(Keys.END).perform()
        time.sleep(1)  # 等待页面加载 

    selector = Selector(text=driver.page_source)
    content_url.extend(selector.xpath(
        '//*[@id="TopicMain"]//div/h2/div/a/@href | //*[@id="TopicMain"]//div/h2/span/a/@href').extract())
    content_url = list(set(content_url))

    # 保存进度
    with open(progress_file, 'w') as f:
        f.write(url)

    print('已经获取' + str(len(content_url)) + '个URL')

    with open('content_url.pkl', 'wb') as f:
        pickle.dump(content_url, f)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF644517012+3522402]
	(No symbol) [0x00007FF644138352]
	(No symbol) [0x00007FF643FE5ABB]
	(No symbol) [0x00007FF643FC287C]
	(No symbol) [0x00007FF644055D97]
	(No symbol) [0x00007FF64406B3CF]
	(No symbol) [0x00007FF64404EE03]
	(No symbol) [0x00007FF64401F4D4]
	(No symbol) [0x00007FF6440205F1]
	GetHandleVerifier [0x00007FF644549B9D+3730157]
	GetHandleVerifier [0x00007FF64459F02D+4079485]
	GetHandleVerifier [0x00007FF6445975D3+4048163]
	GetHandleVerifier [0x00007FF64426A649+718233]
	(No symbol) [0x00007FF644144A3F]
	(No symbol) [0x00007FF64413FA94]
	(No symbol) [0x00007FF64413FBC2]
	(No symbol) [0x00007FF64412F2E4]
	BaseThreadInitThunk [0x00007FFE1E7D8D17+23]
	RtlUserThreadStart [0x00007FFE1F8A7450+32]


In [ ]:
with open('content_url.pkl', 'rb') as f:
    content_url = pickle.load(f)

content_url = ['https:' + url for url in content_url]

with open('content_url.pkl', 'wb') as f:
    pickle.dump(content_url, f)
    

In [2]:
from selenium.common.exceptions import WebDriverException
try:
    
    with open('content_url.pkl', 'rb') as f:
        content_url = pickle.load(f)
    
    # 保存进度的文件名
    progress_file = '../data/progress_get_content_detail.txt'
    
    # 读取进度
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            start_url = f.read().strip()
    else:
        start_url = None
    
    # 打开CSV文件
    with open('zhihu.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for url in content_url:
            if start_url is not None:
                if url == start_url:
                    start_url = None
                continue
    
            if url.startswith('https://www.zhihu.com/question'):
                driver.get(url)
                for _ in range(2):
                    try:
                        target_element = driver.find_element('xpath', '//*[@id="root"]/div/main//div[1]/div/div[4]/a')
            
                        action = ActionChains(driver)
                        action.move_to_element(target_element).perform()
            
                        target_element.click()
            
                    except NoSuchElementException as e:
                        pass
    
                    for i in range(2):
                        # 下滑操作
                        for _ in range(4):
                            actions = ActionChains(driver)
                            actions.send_keys(Keys.END).perform()
                            time.sleep(0.5)
                        # 上滑操作
                        driver.execute_script("window.scrollBy(0, -100);")
    
                selector = Selector(text=driver.page_source)
    
                text_list = selector.xpath('//*[@id="root"]/div/main//span[1]/div/div/span')
    
                for text in text_list:
                    content_list = []
                    content = text.xpath('./p/text()').extract()
                    for i in content:
                        content_list.append(i.strip())
                    f.write(''.join(content_list) + '\n')
    
            elif url.startswith('https://zhuanlan.zhihu.com'):
                driver.get(url)
                for _ in range(5):
                    actions = ActionChains(driver)
                    actions.send_keys(Keys.END).perform()
                    time.sleep(0.5)
    
                selector = Selector(text=driver.page_source)
    
                text_list = selector.xpath('//*[@id="root"]/div/main/div/article/div[1]/div/div/div/p//text()').extract()
    
                text_list = [text.strip() for text in text_list]
    
                f.write(''.join(text_list) + '\n')
    
            else:
                continue
            
            # 保存进度
            with open(progress_file, 'w') as pf:
                pf.write(url)
except WebDriverException as e:
    print(e)
    driver.refresh()

ModuleNotFoundError: No module named 'selenium'